### Crop row application

In [1]:
# Imports
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load image
image1 = cv2.imread("../../data/crop_field_1.jpg")
image2 = cv2.imread("../../data/crop_field_2.jpg")
image = image2

In [3]:
# Grayscale Transform
b, g, r = cv2.split(image)
gray = 2 * g - r - b

ValueError: not enough values to unpack (expected 3, got 0)

In [ ]:
# Skeletonization
size = np.size(gray)
skel = np.zeros(gray.shape, np.uint8)

# Threshold
ret, image_edit = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Skeletonize
element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
done = False
while not done:
    eroded = cv2.erode(image_edit, element)
    temp = cv2.dilate(eroded, element)
    temp = cv2.subtract(image_edit, temp)
    skel = cv2.bitwise_or(skel, temp)
    image_edit = eroded.copy()
    zeros = size - cv2.countNonZero(image_edit)
    if zeros == size:
        done = True

# Shop
plt.imshow(skel, cmap='gray')
plt.show()


In [ ]:
# Hough params
HOUGH_RHO = 2  # Distance resolution of the accumulator in pixels
HOUGH_ANGLE = math.pi * 10.0 / 180  # Angle resolution of the accumulator in radians
HOUGH_THRESHOLD = 30 # Value from which the line is selected in the accumulator

# Detect lines
crop_line_data_1 = cv2.HoughLines(skel, HOUGH_RHO, HOUGH_ANGLE, HOUGH_THRESHOLD)

# Plot lines
for line in crop_line_data_1:
    theta = line[0][1]
    rho = line[0][0]
    a = math.cos(theta)
    b = math.sin(theta)
    x0 = a * rho
    y0 = b * rho
    point1 = (int(round(x0 + 1000 * (-b))), int(round(y0 + 1000 * a)))
    point2 = (int(round(x0 - 1000 * (-b))), int(round(y0 - 1000 * a)))
    cv2.line(image, point1, point2, (0, 0, 255), 2)

plt.imshow(image)
plt.show()